In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.font_manager
from pyod.models.abod import ABOD
from pyod.models.knn import KNN
from sklearn.preprocessing import LabelEncoder

In [13]:
col_name = ['user', 'title', 'rate']
df = pd.read_csv('watchaFinal0220.csv', encoding = 'utf-8-sig', sep = '\t', header = None, names = col_name)
df.head()

,user,title,rate
0,encore01,완득이,4.0
1,encore01,"스물아홉 생일, 1년 후 죽기로 결심했다",5.0
2,encore01,청소부 밥,5.0
3,encore01,말의 품격,5.0
4,encore01,예감은 틀리지 않는다,4.0


### 결측치 확인

In [20]:
df.isna().sum()

user      0
title    10
rate     16
dtype: int64

In [15]:
i, j = np.where(df.isnull().values)

In [18]:
list(zip(df.index[i], df.columns[j])) 

[(228, 'rate'),
 (229, 'rate'),
 (230, 'rate'),
 (231, 'rate'),
 (232, 'rate'),
 (233, 'rate'),
 (234, 'rate'),
 (235, 'rate'),
 (236, 'rate'),
 (237, 'rate'),
 (238, 'rate'),
 (239, 'rate'),
 (240, 'rate'),
 (241, 'rate'),
 (242, 'rate'),
 (243, 'rate'),
 (21009, 'title'),
 (23305, 'title'),
 (24564, 'title'),
 (29071, 'title'),
 (31226, 'title'),
 (38970, 'title'),
 (43045, 'title'),
 (43373, 'title'),
 (46372, 'title'),
 (49124, 'title')]

In [26]:
df.loc[228:243]

,user,title,rate
228,encore05,살인자의 기억법 3.0,NaN
229,encore05,무소유 4.5,NaN
230,encore05,그리고 아무도 없었다 4.0,NaN
231,encore05,오리엔트 특급살인 3.5,NaN
232,encore05,가면 산장 살인 사건 4.0,NaN
233,encore05,너의 췌장을 먹고 싶어 4.0,NaN
234,encore05,오페라의 유령 3.5,NaN
235,encore05,난장이가 쏘아올린 작은 공 4.0,NaN
236,encore05,남한산성 4.5,NaN
237,encore05,냉정과 열정사이 Rosso 4.0,NaN


In [21]:
# 공백제거
encore05Title= df[df['user']=='encore05']['title'].str[:-3].str.strip()

In [22]:
encore05Rate= df[df['user']=='encore05']['title'].str[-3:]
encore05Rate

228    3.0
229    4.5
230    4.0
231    3.5
232    4.0
233    4.0
234    3.5
235    4.0
236    4.5
237    4.0
238    5.0
239    4.0
240    4.0
241    4.0
242    4.0
243    4.0
Name: title, dtype: object

In [23]:
Title05 = pd.DataFrame(encore05Title)

In [24]:
Rate05 = pd.DataFrame(encore05Rate)

In [42]:
Rate05 = Rate05.rename(columns = {'title' : 'rate'})

In [43]:
# 타입 float으로 바꿔주기
Rate05['rate']=Rate05['rate'].astype(float)

In [27]:
Title05

,title
228,살인자의 기억법
229,무소유
230,그리고 아무도 없었다
231,오리엔트 특급살인
232,가면 산장 살인 사건
233,너의 췌장을 먹고 싶어
234,오페라의 유령
235,난장이가 쏘아올린 작은 공
236,남한산성
237,냉정과 열정사이 Rosso


In [44]:
Rate05

,rate
228,3.0
229,4.5
230,4.0
231,3.5
232,4.0
233,4.0
234,3.5
235,4.0
236,4.5
237,4.0


In [38]:
df.loc[228:243]['title'] = Title05['title']

C:\Users\Playdata\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
df[df['user']=='encore05']

,user,title,rate
228,encore05,살인자의 기억법,NaN
229,encore05,무소유,NaN
230,encore05,그리고 아무도 없었다,NaN
231,encore05,오리엔트 특급살인,NaN
232,encore05,가면 산장 살인 사건,NaN
233,encore05,너의 췌장을 먹고 싶어,NaN
234,encore05,오페라의 유령,NaN
235,encore05,난장이가 쏘아올린 작은 공,NaN
236,encore05,남한산성,NaN
237,encore05,냉정과 열정사이 Rosso,NaN


In [45]:
df.loc[228:243]['rate'] = Rate05['rate']

C:\Users\Playdata\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
df[df['user']=='encore05']

,user,title,rate
228,encore05,살인자의 기억법,3.0
229,encore05,무소유,4.5
230,encore05,그리고 아무도 없었다,4.0
231,encore05,오리엔트 특급살인,3.5
232,encore05,가면 산장 살인 사건,4.0
233,encore05,너의 췌장을 먹고 싶어,4.0
234,encore05,오페라의 유령,3.5
235,encore05,난장이가 쏘아올린 작은 공,4.0
236,encore05,남한산성,4.5
237,encore05,냉정과 열정사이 Rosso,4.0


In [ ]:
# 나머지 결측치들 삭제하기
 (21009, 'title'),
 (23305, 'title'),
 (24564, 'title'),
 (29071, 'title'),
 (31226, 'title'),
 (38970, 'title'),
 (43045, 'title'),
 (43373, 'title'),
 (46372, 'title'),
 (49124, 'title')]

In [54]:
df02 = df.dropna()

In [55]:
df02.isna().sum()

user     0
title    0
rate     0
dtype: int64

In [57]:
df02.to_csv('watchaFinal0220.csv', encoding = 'utf-8-sig', sep = '\t', header = None, index = False)

In [59]:
df03 = pd.read_csv('watchaFinal0220.csv', encoding = 'utf-8-sig', sep = '\t', header = None)

In [61]:
df03.isna().sum()

0    0
1    0
2    0
dtype: int64

In [60]:
df03[df03[0] == 'encore05']

,0,1,2
228,encore05,살인자의 기억법,3.0
229,encore05,무소유,4.5
230,encore05,그리고 아무도 없었다,4.0
231,encore05,오리엔트 특급살인,3.5
232,encore05,가면 산장 살인 사건,4.0
233,encore05,너의 췌장을 먹고 싶어,4.0
234,encore05,오페라의 유령,3.5
235,encore05,난장이가 쏘아올린 작은 공,4.0
236,encore05,남한산성,4.5
237,encore05,냉정과 열정사이 Rosso,4.0
